# Self Driving Traffic Sign Detection

Isaiah Jenkins

## Import the required libraries

In [2]:
import tensorflow as tf
from keras import layers, Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import ResNet50

from sklearn.model_selection import train_test_split

import pandas as pd
import os

## 1. About the data

1. a. Description

Throughout this analysis we will explore Udacity's Self Driving dataset. This dataset consists of images for thousands of pedestrians, bikers, cars, and traffic lights. Although traffic light images are underrepresented in the dataset, the focus of the analysis will be based solely on traffic light images.

1. b. Data dictionary, 97,942 labels across 11 classes and 15,000 images, 1,720 null examples (images with no labels).

Class Balance across images (labels in each image)

* car - 64,399 - over represented
* pedestrian - 10,806
* trafficLight-Red - 6,870
* trafficLight-Green - 5,465 - under represented
* truck - 3,623 - under represented
* trafficLight - 2,568 - under represented
* biker - 1,864 - under represented
* trafficLight-RedLeft - 1,751 - under represented
* trafficLight-GreenLeft - 310 - under represented
* trafficLight-Yellow - 272 - under represented
* trafficLight-YellowLeft - 14 - under represented

## 2. Objectives

Throughout this analysis, we will explore and build various deep learning convolutional neural network (CNN) architectures to detect traffic light signs, aiming to optimize accuracy and efficiency. Our objective is to compare different model variations, such as CNNs with different depths, pre-trained models, and data augmentation techniques, to determine the most effective approach. Potential challenges include handling variations in lighting conditions, occlusions, and small object sizes, which may impact detection performance. Additionally, dataset imbalances and misclassifications due to similar-looking traffic signs could introduce biases, requiring careful preprocessing and model tuning.

## 3. Data Exploration, Cleaning and Feature Engineering

* Extract traffic sign & traffic light images from the dataset to help with computational efficiency.
* Resize images to standardize dataset making it computationally efficient making it easier to analyze data.
* Normalize pixel values (0-1 range) to help with generalization.
* Augment data (rotation, brightness shifts, contrast changes) to improve generalization.

### Load in data

In [3]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

!unzip -o -q '/content/gdrive/MyDrive/data.zip' -d '/content/'

Mounted at /content/gdrive


In [4]:
DATASET_PATH = 'data/export'
CSV_PATH = os.path.join(DATASET_PATH, '_annotations.csv')

In [5]:
df = pd.read_csv(CSV_PATH) # load annotations

In [6]:
df = df[df['class'].isin(['trafficLight-Red', 'trafficLight_Yellow', 'trafficLight-Green'])]

In [7]:
file_names = df['filename'].values # image files names
labels = df['class'].values # classes

In [8]:
label_map = {"trafficLight-Red": 0, "trafficLight_Yellow": 1, "trafficLight-Green": 2, }  # Adjust based on dataset
labels = [label_map[label] for label in labels if label in label_map]  # Convert text labels to integers
labels = to_categorical(labels, num_classes=3)  # Convert to one-hot encoding

In [9]:
#Split Data into Train (80%) and Test (20%)
train_files, test_files, train_labels, test_labels = train_test_split(
    file_names, labels, test_size=0.2, random_state=42, stratify=labels, shuffle=True
)

In [10]:
IMG_SIZE = (224, 224)

In [11]:
# Function to Load & Preprocess Images
def load_and_preprocess_image(file_name, label):
    img_path = os.path.join(DATASET_PATH, file_name)  # Update image folder path
    img = load_img(img_path, target_size=IMG_SIZE)  # Load & Resize Image
    img = img_to_array(img) / 255.0  # Convert to NumPy array & Normalize (0-1)
    return img, label

In [12]:
# Create Generators for Train & Test Data
def data_generator(file_list, label_list):
    for f, l in zip(file_list, label_list):
        yield load_and_preprocess_image(f, l)  # Load one image at a time

In [13]:
BATCH_SIZE = 10

In [14]:
# Create a tf.data.Dataset that Loads Images on Demand
train_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(train_files, train_labels),
    output_signature=(
        tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32),  # Image Shape
        tf.TensorSpec(shape=(3,), dtype=tf.float32)  # One-hot Encoded Label
    )
).batch(BATCH_SIZE).shuffle(1000).prefetch(tf.data.experimental.AUTOTUNE)

test_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(test_files, test_labels),
    output_signature=(
        tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32),  # Image Shape
        tf.TensorSpec(shape=(3,), dtype=tf.float32)  # One-hot Encoded Label
    )
).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

## 4. CNN Models

#### ResNet-50

In [16]:
# Layers
num_classes = 3

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dense(1024, activation='relu')(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [17]:
# Compile
model = Model(inputs=base_model.input, outputs=outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 230, 230, 3)    │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 56, 56, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├──────────────────────

 Total params: 25,688,963 (98.00 MB)

 Trainable params: 25,635,843 (97.79 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [18]:
# Fit on training set and compare against validation set after each epoch
model.fit(train_dataset, epochs=5, validation_data=test_dataset)

Epoch 1/5
   1961/Unknown 344s 131ms/step - accuracy: 0.7057 - loss: 0.6208

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


1961/1961 ━━━━━━━━━━━━━━━━━━━━ 366s 142ms/step - accuracy: 0.7057 - loss: 0.6207 - val_accuracy: 0.6174 - val_loss: 0.8490
Epoch 2/5
1961/1961 ━━━━━━━━━━━━━━━━━━━━ 270s 126ms/step - accuracy: 0.8758 - loss: 0.3123 - val_accuracy: 0.8472 - val_loss: 0.3760
Epoch 3/5
1961/1961 ━━━━━━━━━━━━━━━━━━━━ 271s 126ms/step - accuracy: 0.9260 - loss: 0.2085 - val_accuracy: 0.8786 - val_loss: 0.5685
Epoch 4/5
1961/1961 ━━━━━━━━━━━━━━━━━━━━ 270s 126ms/step - accuracy: 0.9444 - loss: 0.1595 - val_accuracy: 0.8113 - val_loss: 0.5056
Epoch 5/5
1961/1961 ━━━━━━━━━━━━━━━━━━━━ 270s 126ms/step - accuracy: 0.9556 - loss: 0.1302 - val_accuracy: 0.9211 - val_loss: 0.2447


In [19]:
# Evaluate - Loss, Accuracy
model.evaluate(test_dataset)

491/491 ━━━━━━━━━━━━━━━━━━━━ 16s 32ms/step - accuracy: 0.9268 - loss: 0.2311


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


[0.244716078042984, 0.9210687279701233]

In [3]:
# Inception

In [ ]:
# Custom CNN 1

### Summary of Models

## 5. Insights and Key Findings

## 6. Next Steps